# Face Mask Detection

Masks play an important role in protecting people's health from respiratory diseases, as they are one of the few precautions available for COVID-19 in the absence of immunization. Using this NoteBook, a model can be created to identify people who use masks, do not use them, or use inappropriate masks.


### Table of Contents:

1. [Introduction](#Introduction)
2. [Algorithm used](#Algorithm-used)
3. [Import libraries](#Import-libraries)
4. [Call data set](#call-data-set)
5. [Read all photos](#read-all-photos)
6. [Specify the output](#specify-the-output)
7. [Normalization](#normalization)
8. [Training and test data](#training-and-test-data)
9. [create Model](#create-model)
10. [compile model](#compile-model)
11. [Model testing](#model-testing)
12. [Save Model](#save-model)

### Introduction:

The purpose of this notebook is to build a face mask recognition model and how to implement this model.

### Algorithm used:





`CNN(Convolutional Neural Network)` algorithm is used in this code, which is one of the most powerful deep learning algorithms.

A Convolutional Neural Network (CNN), also known as ConvNet, is a specialized type of deep learning algorithm mainly designed for tasks that necessitate object recognition, including image classification, detection, and segmentation. CNNs are employed in a variety of practical scenarios, such as autonomous vehicles, security camera systems, and others.

### Import libraries

In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten,Dropout,MaxPool2D
from sklearn.model_selection import train_test_split
import tensorflow
import pickle

### Call data set

**Hint:** This data set is called in colab, you must change the path as below.

path_with_mask: `../DataSets/with_mask`
path_without_mask: `../DataSets/without_mask`

To read the datasets, we used `os.listdir`, which is one of Python's internal methods and libraries.

In [3]:
img_path_with_mask=os.listdir('/content/drive/MyDrive/with_mask')
img_path_without_mask=os.listdir('/content/drive/MyDrive/without_mask')

In [4]:
# Set weight & height (w,h)
w,h=100,100

### Read all photos

We have read the photos using `cv2.imread` and then add them all to a list to have a list of all photos array.

All photos must be converted to the same size to be used in the algorithm, and for this we use `cv2.resize`.

**Hint:** This data set is called in colab, you must change the path as below.

path_with_mask: `../DataSets/with_mask`
path_without_mask: `../DataSets/without_mask`

In [5]:
mask=[]
for i in img_path_with_mask:
    img=cv2.imread('/content/drive/MyDrive/with_mask/'+i)
    img=cv2.resize(img,(w,h))
    mask.append(img)

In [6]:
mask=np.array(mask)
mask.shape

(2892, 100, 100, 3)

In [7]:
nomask=[]
for i in img_path_without_mask:
  img=cv2.imread('/content/drive/MyDrive/without_mask5/'+i)
  img=cv2.resize(img,(w,h))
  nomask.append(img)

In [8]:
nomask=np.array(nomask)
nomask.shape

(3476, 100, 100, 3)

### Specify the output

`1` for people who wore masks and `0` for those who did not
For this, we used two methods `np.ones` and `np.zeros`, and then we combined the two presentations with the `np.concatenate` method and turned them into categorical using the `to_categorical` method.

In [9]:
result_nomask=np.zeros(nomask.shape[0])
result_mask=np.ones(mask.shape[0])
result=np.concatenate((result_nomask,result_mask))
result_cat = to_categorical(result, num_classes=2)

### Normalization

For this, we used the following formula:

$$ \frac{array-min(array)}{max(array) - min(array)} $$

In [10]:
mask=mask.reshape(mask.shape[0], w,h, 3)
nomask=nomask.reshape(nomask.shape[0], w,h, 3)
mask=(mask-mask.min())/(mask.max()-mask.min())
nomask=(nomask-nomask.min())/(nomask.max()-nomask.min())

In [11]:
# mix 2 Array
input_dl=np.concatenate((nomask,mask))

### Training and test data

For this, we used the `train_test_split` method, with the use of this method, the data is divided for us in a shuffled manner, and we can use the test data provided by us to test our model.

In [12]:
# creat test & train data
x_train, x_test, y_train, y_test=train_test_split(input_dl, result_cat, test_size=.2, random_state=1000)

### create Model

We used `Sequential` to build the `model` and we used the `Conv2D` method to add c`onvolution layers` and set the required values such as the `kernel size`, etc. and we used the `MaxPool2D` method for the `pool layers` and the `Dropout` method for the layer We used `Dropout` and we used `Dense` for `fully connected layers` and we used two activation functions `relu` and `sigmoid` to activate the functions.

In [13]:
model = Sequential()

model.add(Conv2D(25, kernel_size=5,strides=(1,1),padding='same',
                 activation='relu', input_shape=(100,100, 3)))
model.add(Conv2D(50,strides=(1,1),padding='same', kernel_size=5,
                 activation='relu'))
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(75,strides=(1,1),padding='same', kernel_size=3,
                 activation='relu'))

model.add(MaxPool2D(pool_size=(1,1)))
model.add(Dropout(0.25))
model.add(Conv2D(125,strides=(1,1),padding='same', kernel_size=3,
                 activation='relu'))
model.add(MaxPool2D(pool_size=(1, 1)))
model.add(Conv2D(175,strides=(1,1),padding='same', kernel_size=3,
                 activation='relu'))
model.add(MaxPool2D(pool_size=(1, 1)))
model.add(Dropout(rate=0.25))
model.add(Flatten())

model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### compile model

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callback = tensorflow.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.01,
    patience=2
)
model.fit(x=x_train, y=y_train,
          batch_size=100, epochs=20,
         validation_data=(x_test, y_test)
          ,callbacks=[callback]
         )

Epoch 1/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 917s 18s/step - accuracy: 0.6650 - loss: 0.6001 - val_accuracy: 0.9584 - val_loss: 0.1255
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 919s 18s/step - accuracy: 0.9630 - loss: 0.1285 - val_accuracy: 0.9694 - val_loss: 0.0835
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 940s 18s/step - accuracy: 0.9717 - loss: 0.1021 - val_accuracy: 0.9796 - val_loss: 0.0797
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 930s 18s/step - accuracy: 0.9722 - loss: 0.0996 - val_accuracy: 0.9733 - val_loss: 0.0877
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 978s 18s/step - accuracy: 0.9792 - loss: 0.0844 - val_accuracy: 0.9851 - val_loss: 0.0487


### Model testing


We used `evaluate` method and `test data` to test the model

In [28]:
model.evaluate(x_test, y_test)

40/40 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.9869 - loss: 0.0465


[0.04865759238600731, 0.9850863218307495]

### Save Model

Getting the output from the model using the `pickle.dump` method.

**Hint:** In order for the code to work fully with the `Django` site, you must place the created model in the directory of the `Django` project (`DL_Detection`) in the `model` directory.

If the model directory was not created, create it in `BASE_DIR`.

In [ ]:
pickle.dump(model, open('model_9850_0486.pkl', 'wb'))